In [21]:
import pandas as pd
import cpi
from skimpy import clean_columns
from datetime import date

In [3]:
cpi.update()

In [22]:
df = pd.read_csv('merged_file.csv')

In [23]:
# clean columns
df = clean_columns(df)

11 column names have been cleaned

In [24]:
# Convert the 'Closing Date' column to datetime
df['Year'] = pd.to_datetime(df['closing_date'], format='%d-%b-%y')

In [27]:
# Convert the 'Closing Date' column to datetime
df['Year'] = pd.to_datetime(df['Year'], format='%Y')

In [28]:
# removing the $ from "Asset" and "Deposite" columns
df['approx_asset_millions'] = df['approx_asset_millions'].str.replace('$','').str.replace(',','').str.replace('.','')
df['approx_deposit_millions'] = df['approx_deposit_millions'].str.replace('$','').str.replace(',','').str.replace('.','')

/var/folders/yp/8z8rglc91l55828y91ptx4dr0000gn/T/ipykernel_76329/2916092983.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['approx_asset_millions'] = df['approx_asset_millions'].str.replace('$','').str.replace(',','').str.replace('.','')
/var/folders/yp/8z8rglc91l55828y91ptx4dr0000gn/T/ipykernel_76329/2916092983.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['approx_deposit_millions'] = df['approx_deposit_millions'].str.replace('$','').str.replace(',','').str.replace('.','')


In [29]:
# Convert the 'Asset' and 'Deposit' columns to integers
df['approx_asset_millions'] = df['approx_asset_millions'].astype(int)
df['approx_deposit_millions'] = df['approx_deposit_millions'].astype(int)

In [30]:
# only the year from "Year"
df['year'] = df['Year'].apply(lambda x: pd.to_datetime(x).year)


In [31]:
# Create new columns for inflation-adjusted 'Deposit' values
def inflate_asset(row):
    if row['Year'].year == 2023:
        return row['approx_deposit_millions']
    else:
        try:
            return cpi.inflate(row['approx_deposit_millions'], row['Year'].year)
        except CPIObjectDoesNotExist:
            return np.nan

df['Inflation-Adjusted_Deposite'] = df.apply(inflate_asset, axis=1)


In [32]:
# Create new columns for inflation-adjusted 'Assets' values
def inflate_asset(row):
    if row['Year'].year == 2023:
        return row['approx_asset_millions']
    else:
        try:
            return cpi.inflate(row['approx_asset_millions'], row['Year'].year)
        except CPIObjectDoesNotExist:
            return np.nan

df['Inflation-Adjusted_Asset'] = df.apply(inflate_asset, axis=1)


In [35]:
df.to_csv('inflation_adjusted.csv', index=False)